In [1]:
# !python -m pip install --upgrade --user ortools

# import nltk

# nltk.download('punkt')
# nltk.download('stopwords')

In [2]:
# from google.colab import files
# files.upload()

In [3]:
import numpy as np
import pandas as pd
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import nltk
import string
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from numpy import linalg as LA
import math

from ortools.linear_solver import pywraplp

In [4]:
data = pd.read_csv('data.csv')
print(data.shape)
data.head(2)

(743, 43)


,X.AUTHID,anger_doc_sum,anticipation_doc_sum,arousal_doc_sum,disgust_doc_sum,dominance_doc_sum,fear_doc_sum,joy_doc_sum,neg_doc_sum,num_text_words,...,cSIN_label_fin,cEXC_label_fin,cCOM_label_fin,cRUG_label_fin,cSOP_label_fin,cSIN_fin_rank,cEXC_fin_rank,cCOM_fin_rank,cRUG_fin_rank,cSOP_fin_rank
0,b'http://blog.searsholdings.com/leadership-vie...,1,9,31.317,0,38.293,7,16,7,124,...,n,y,y,n,y,5.0,1.0,2.0,3.0,4.0
1,b'https://atyourservice.blogs.xerox.com/catego...,0,4,23.260,0,32.665,0,1,2,126,...,y,n,y,n,y,NaN,NaN,NaN,NaN,NaN


In [5]:
site_content = data['site.content']
docs = {}
for i in range(site_content.size):
    docs[i] = eval(site_content[i]).decode('utf-8')

In [6]:
original_corpus = [1]*len(docs)
corpus = [1]*len(docs)
copy_corpus = [1]*len(docs)

for i in range(len(docs)):
    original_corpus[i] = sent_tokenize(docs[i])
    corpus[i] = sent_tokenize(docs[i])
    copy_corpus[i] = sent_tokenize(docs[i])

In [7]:
print(site_content[20])
print(docs[20])
print(corpus[20])

b"Join me! Ill be hosting a unique set of customersat ;, SYN109, (Wednesday May 24, 4:45pm) where well have a discussion and audience-driven Q&A with 4 Citrix Cloud customers:;Well cut the marketing fluff. No speeds and feeds. Just candid, matter-of-fact discussions about:;Ken is Sr. Director of Product Marketing in Citrix's Cloud Services group, and spends most of his time helping lead the charge to converge concepts of cloud computing, virtual workspaces, and hosted service providers. You can also find Ken on Twitter as @fountnhead; Weve seen increasing traction for NetScaler in customer deployments as they migrate to the cloud. One of the key differentiators ; Citrix VP of WW Sales and Strategy, Ash Vijayakanthan shares his thoughts on the benefits Citrix Cloud brings for Citrix Service Providers ; NetScaler Management and Analytics from Citrix allows administrators to easily view, automate, and manage all their application infrastructure, as well ;Dont worry, you can unsubscribe at

In [8]:
stemmer = nltk.stem.porter.PorterStemmer()
lemmatizer = nltk.stem.WordNetLemmatizer()
remove_punctuation_map = dict((ord(char), ' ') for char in string.punctuation)
integer_chars = [str(digit) for digit in list(range(10))]
remove_integer_map = dict((ord(char), None) for char in integer_chars)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

def lemm_tokens(tokens):
    return [lemmatizer.lemmatize(item) for item in tokens]

def filtersent(text):
    # remove puctuationas and lower the case
    simpletext = text.lower().translate(remove_punctuation_map).translate(remove_integer_map)
    # tokenize
    words = nltk.word_tokenize(simpletext)
    # remove stop words
    words = [w for w in words if w not in stopwords.words('english')]
    # stem them
    words = lemm_tokens(words)
    words = stem_tokens(words)
    words = [w for w in words if w not in string.ascii_lowercase]
    return TreebankWordDetokenizer().detokenize(words)

In [9]:
for i in range(len(corpus)):
    for j in range(len(corpus[i])):
        corpus[i][j] = filtersent(original_corpus[i][j])
        copy_corpus[i][j] = original_corpus[i][j]

In [10]:
for i in range(len(corpus)):
    idx = []
    for j in range(len(corpus[i])):
        if(corpus[i][j] == ''):
            idx.insert(0,j)
    for index in idx:
        del corpus[i][index]
        del copy_corpus[i][index]

In [11]:

# print(original_corpus[498])
# print(corpus[498])
# print(copy_corpus[498])
# print(simplefiltersent("Join me! Ill be hosting a unique set of customersat"))

In [12]:
print(corpus[20])
print(copy_corpus[20])
print(original_corpus[20])

['join', 'ill host uniqu set customersat syn wednesday may pm well discuss audienc driven citrix cloud custom well cut market fluff', 'speed feed', 'candid matter fact discuss ken sr director product market citrix cloud servic group spend time help lead charg converg concept cloud comput virtual workspac host servic provid', 'also find ken twitter fountnhead weve seen increas traction netscal custom deploy migrat cloud', 'one key differenti citrix vp ww sale strategi ash vijayakanthan share thought benefit citrix cloud bring citrix servic provid netscal manag analyt citrix allow administr easili view autom manag applic infrastructur well dont worri unsubscrib anytim']
['Join me!', 'Ill be hosting a unique set of customersat ;, SYN109, (Wednesday May 24, 4:45pm) where well have a discussion and audience-driven Q&A with 4 Citrix Cloud customers:;Well cut the marketing fluff.', 'No speeds and feeds.', "Just candid, matter-of-fact discussions about:;Ken is Sr. Director of Product Marketing

In [13]:
tf_word = [1]*len(corpus)
idf_word = [1]*len(corpus)
tfidf_word = [1]*len(corpus)

# calculating only tf in each doc
for i in range(len(corpus)):
    sentence = TreebankWordDetokenizer().detokenize([string for string in corpus[i]])
    
    words = nltk.tokenize.word_tokenize(sentence)
    fdist1 = nltk.FreqDist(words)

    tf_word[i] = dict((word, freq) for word, freq in fdist1.items() if not word.isdigit())

In [14]:
for i in range(len(corpus)):
    idf_word[i] = tf_word[i]
    tfidf_word[i] = tf_word[i]

# calculating idf
for i in range(len(corpus)):
    for word in idf_word[i]:
        df=0 #documents containg the word
        for p in range(len(corpus)):
            if word in tf_word[p]:
                df += 1;
        idf_word[i][word] = np.log10(len(corpus)/(df + 1))

# calculating tfidf
for i in range(len(corpus)):
    for word in tfidf_word[i]:
        tfidf_word[i][word] = tf_word[i][word] * idf_word[i][word]

In [15]:
tfidf_sent = [1]*len(corpus)

for i in range(len(corpus)):
    tfidf_sent[i] = [1]*len(corpus[i])

for i in range(len(corpus)):
    for j in range(len(corpus[i])):
        tfidf = 0
        for word in word_tokenize(corpus[i][j]):
            try:
                tfidf += tfidf_word[i][word]
            except:
                pass
        if(len(corpus[i][j])==0):
            tfidf = 0
        else:
            tfidf /= len(corpus[i][j])
        tfidf_sent[i][j] = tfidf

In [16]:
# count matrix
count_word = [1]*len(corpus)
count_matrix = [1]*len(corpus)
count_persent = [1]*len(corpus) # no of words per sentence

vectorizer = CountVectorizer()
for i in range(len(corpus)):
    count_word[i] = vectorizer.fit_transform(corpus[i])
    count_matrix[i] = count_word[i].toarray()
    count_persent[i] = np.sum(count_matrix[i], axis=1)

In [17]:
count_persent[20]

array([ 1, 20,  2, 27, 14, 36], dtype=int64)

In [18]:
lam = 0.6
k = 3 # maximum sentences


mu = 0.85
top_corpus = [1]*len(corpus)
corpus_sents = [1]*len(corpus)

for s in range(len(corpus)):
# if(True):
#     s = 498;
    no_sent = len(corpus[s])
    # similarity matrix
    sim = np.zeros((no_sent, no_sent))
    worthless_sent = []
    for x in range(no_sent):
        for y in range(no_sent):
            adotb = np.dot(count_matrix[s][x], count_matrix[s][y])
            a = LA.norm(count_matrix[s][x])
            b = LA.norm(count_matrix[s][y])
            if(x==y):
                sim[x,y] = (1-mu)/no_sent
            else:
                sim[x,y] = mu*(adotb/(a*b)) + (1-mu)/no_sent

    denom = np.sum(sim,axis=1)
    for x in range(no_sent):
        sim[x,:] = sim[x,:] * 1/denom[x]
        
    v = np.array(tfidf_sent[s])
    
    while(True):
        v_dash = v.dot(sim)
        if(LA.norm(v_dash - v) < 0.000000001):
            break
        v = v_dash
        
    alpha = {}
    beta = {}
    redundacy = sim;

    solver = pywraplp.Solver('SolveAssignmentProblemMIP', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

    for i in range(no_sent):
        alpha[i] = solver.BoolVar('alpha[%i]' % i)

    for i in range(no_sent):
        for j in range(i+1,no_sent):
            beta[i,j] = solver.BoolVar('beta[%i,%i]' %(i,j))


    solver.Maximize(solver.Sum(alpha[i]*lam*v[i] for i in range(no_sent)) - solver.Sum([beta[i,j]*(1-lam)*redundacy[i,j] for j in range(i+1,no_sent) for i in range(no_sent)]))

    # constraints
    solver.Add( solver.Sum([alpha[i] for i in range(no_sent)]) <= k )

    for i in range(no_sent):
        for j in range(i+1,no_sent):
            solver.Add( beta[i,j] <= alpha[i])
            solver.Add( beta[i,j] <= alpha[j])
            solver.Add( alpha[i] + alpha[j] <= 1 + beta[i,j])



    sol = solver.Solve()

    # print([alpha[i].solution_value() for i in range(no_sent)])
    # print([[beta[i,j].solution_value() for j in range(i+1, no_sent) ] for i in range(no_sent) ])
    # print(solver.Objective().Value())

    alpha_sol = [alpha[i].solution_value() for i in range(no_sent)]
    top_sents = []
    for i in range(no_sent):
        if alpha_sol[i]==1:
            top_sents.append(copy_corpus[s][i])
    top_corpus[s] = top_sents

In [19]:
top_data = pd.DataFrame({"X.AUTHID": data["X.AUTHID"],
                        "ILP1": [ s[0] for s in top_corpus],
                         "ILP2": [ s[1] for s in top_corpus],
                         "ILP3": [ s[2] for s in top_corpus]})
top_data.head(5)

,X.AUTHID,ILP1,ILP2,ILP3
0,b'http://blog.searsholdings.com/leadership-vie...,SHC Speaks by As Chief HR Officer and co-e spo...,"This year, SHC partnered with , and to marry f...",I was joined by dozens of SHC associates and f...
1,b'https://atyourservice.blogs.xerox.com/catego...,When you look at your Xerox device and see the...,Recently new support videos were posted to the...,Xerox and Xerox and Design are trademarks of X...
2,b'http://blogs.zebra.com/whitepaper-label',"; March 17, 2015 at 8:03 AM ;Product identifi...",It is critical that the labels you utilize are...,";In this paper, Zebra provides guidance on how..."
3,b'https://connect.blogs.xerox.com/author/lauri...,Writer and PR Consultant Email LinkedIn Twitte...,"By By Laurie Riedman, PR Consultant Throughout...",Xerox and Xerox and Design are trademarks of X...
4,b'http://blog.searsholdings.com/2016/07/page/2/',"For our Danbury Sears store, this meant that w...","On Saturday, July 2, Sears celebrated the gran...","With the pride that I have for Sears Holdings,..."


In [20]:
lam = 0.6
k = 3 # maximum sentences


mu = 0.85
top_corpus = [1]*len(corpus)
corpus_sents = [1]*len(corpus)

for s in range(len(corpus)):
# if(True):
#     s = 498
    no_sent = len(corpus[s])
    # similarity matrix
    sim = np.zeros((no_sent, no_sent))
    worthless_sent = []
    for x in range(no_sent):
        for y in range(no_sent):
            adotb = np.dot(count_matrix[s][x], count_matrix[s][y])
            a = LA.norm(count_matrix[s][x])
            b = LA.norm(count_matrix[s][y])
            if(x==y):
                sim[x,y] = (1-mu)/no_sent
            else:
                sim[x,y] = mu*(adotb/(a*b)) + (1-mu)/no_sent

    denom = np.sum(sim,axis=1)
    for x in range(no_sent):
        sim[x,:] = sim[x,:] * 1/denom[x]
        
    v = np.array(tfidf_sent[s])
    
    while(True):
        v_dash = v.dot(sim)
        if(LA.norm(v_dash - v) < 0.000000001):
            break
        v = v_dash
        
    top_sents = []
    for i in range(k):
        idx = np.argmax(v)
        top_sents.append(copy_corpus[s][idx])
        v[idx] = -np.inf
        v = [lam*v[i] - (1-lam)*sim[idx,i] for i in range(no_sent)]
    
    top_corpus[s] = top_sents

In [21]:
top_data["MMR1"] = [s[0] for s in top_corpus]
top_data["MMR2"] = [s[1] for s in top_corpus]
top_data["MMR3"] = [s[2] for s in top_corpus]
                     
top_data.head(5)

,X.AUTHID,ILP1,ILP2,ILP3,MMR1,MMR2,MMR3
0,b'http://blog.searsholdings.com/leadership-vie...,SHC Speaks by As Chief HR Officer and co-e spo...,"This year, SHC partnered with , and to marry f...",I was joined by dozens of SHC associates and f...,"This year, SHC partnered with , and to marry f...",I was joined by dozens of SHC associates and f...,SHC Speaks by As Chief HR Officer and co-e spo...
1,b'https://atyourservice.blogs.xerox.com/catego...,When you look at your Xerox device and see the...,Recently new support videos were posted to the...,Xerox and Xerox and Design are trademarks of X...,Xerox and Xerox and Design are trademarks of X...,Recently new support videos were posted to the...,All rights reserved.
2,b'http://blogs.zebra.com/whitepaper-label',"; March 17, 2015 at 8:03 AM ;Product identifi...",It is critical that the labels you utilize are...,";In this paper, Zebra provides guidance on how...",It is critical that the labels you utilize are...,";In this paper, Zebra provides guidance on how...",This will reduce your risk of non-compliance t...
3,b'https://connect.blogs.xerox.com/author/lauri...,Writer and PR Consultant Email LinkedIn Twitte...,"By By Laurie Riedman, PR Consultant Throughout...",Xerox and Xerox and Design are trademarks of X...,"By By Laurie Riedman, PR Consultant Throughout...",Remembering how when my children were young it...,I was thinking about them today.
4,b'http://blog.searsholdings.com/2016/07/page/2/',"For our Danbury Sears store, this meant that w...","On Saturday, July 2, Sears celebrated the gran...","With the pride that I have for Sears Holdings,...","For our Danbury Sears store, this meant that w...","With the pride that I have for Sears Holdings,...",SHC Speaks by As the retail landscape continue...


In [22]:
# from google.colab import files

top_data.to_csv('topdata_textbase.csv',encoding='utf-8', index=False)
# files.download('topdata.csv')

In [23]:
top_corpus[0]

['This year, SHC partnered with , and to marry four couples on our float during the parade.',
 'I was joined by dozens of SHC associates and family members who marched in the parade or cheered from the crowd.',
 'SHC Speaks by As Chief HR Officer and co-e sponsor of Sears Holdings , I am honored to have volunteered at the 45 Annual Chicago Pride Parade .']